# Phase 1

# 1.Webscrape the provided URL  - IMDB dataset

Importing Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from time import sleep
from random import randint
import numpy as np
import pandas as pd
import csv
import sqlite3

In [1]:
# Sending request to imdb url

In [2]:

url = 'https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1'
response = requests.get(url)
print(response)

<Response [200]>


In [3]:
# scrapping code to collect the data in csv file

Sno=[]
MoviesName=[]
Director_1=[]
Director_2=[]
Director_3=[]
Duration=[]
Metascore=[]
Year=[]
Ratings=[]
   
pages = np.arange(1,1750,50)


for page in  tqdm(pages):
    
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start=" + str(page) + "&ref_=adv_nxt")
  
    soup=BeautifulSoup(page.text, 'html.parser')
  
    movies=soup.find('div',class_="lister-list").find_all('div',class_='lister-item-content')
  
    sleep(2)
    
    for movie in movies:
        sno=movie.find('h3' ,class_="lister-item-header").span.text.strip('.').replace(',','')
        Sno.append(int(sno))
        movie_name=movie.find('h3' ,class_="lister-item-header").a.text
        MoviesName.append(movie_name)
        director=movie.find('p',class_="").get_text(strip=True).split('|')[0]
        if len(director.split(','))==3:
            director_1=director.split(',')[0].strip(' Directors: ')
            director_2=director.split(',')[1]
            director_3=director.split(',')[2]
            Director_1.append(director_1)
            Director_2.append(director_2)
            Director_3.append(director_3)
            
        elif len(director.split(','))==2:
            director_1=director.split(',')[0].strip(' Directors: ')
            director_2=director.split(',')[1]
            Director_1.append(director_1)
            Director_2.append(director_2)
            Director_3.append(np.nan)
            
            
        else:
            director_1=director.strip(' Directors: ')
            Director_1.append(director_1)
            Director_2.append(np.nan)
            Director_3.append(np.nan)
            
        
        duration=movie.find('span',class_="runtime").text.strip(' min')
        Duration.append(duration)
        year=movie.find('span',class_="lister-item-year text-muted unbold").text.strip('(I) ').strip('()')
        Year.append(year)
        ratings=movie.find('div', class_="inline-block ratings-imdb-rating").strong.text.strip('.')
        Ratings.append(ratings)
        try:
            metascore=movie.find('div', class_="inline-block ratings-metascore").span.get_text(strip=True)
            Metascore.append(metascore)
        except Exception as e:
            metascore=np.nan
            Metascore.append(metascore)
        
print(Sno,MoviesName,Director_1,Director_2,Director_3,Duration,Year,Ratings,Metascore)

100%|██████████| 35/35 [01:56<00:00,  3.34s/it]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [4]:
# check the length of  collected data
print(len(Sno))

1749


# 2. Store the entire data in two different CSV files 

Creating CSV file-1

In [9]:
# create the csv file
with open ("D:\Internship\Capstone Project\movie_1.csv",'w', newline='', encoding='utf-8') as f:
    writer=csv.DictWriter(f, fieldnames=['Serial no','Movie Name','Director_1','Director_2','Director_3','Duration','Year','Ratings','Metascore'])
    writer.writeheader()
    
    for Sn,Mn,D1,D2,D3,D,Y,R,M in zip(Sno,MoviesName,Director_1,Director_2,Director_3,Duration,Year,Ratings,Metascore):
        writer.writerow({'Serial no':Sn, 'Movie Name':Mn, 'Director_1':D1,'Director_2':D2,'Director_3':D3,'Duration':D, 'Year':Y,'Ratings':R,'Metascore':M})
                         

In [11]:
# read the data in csv 
df=pd.read_csv("D:\Internship\Capstone Project\movie_1.csv")
df

,Serial no,Movie Name,Director_1,Director_2,Director_3,Duration,Year,Ratings,Metascore
0,1,The Dark Knight,Christopher Nolan,NaN,NaN,152,2008,9.0,84.0
1,2,The Lord of the Rings: The Return of the King,Peter Jackson,NaN,NaN,201,2003,9.0,94.0
2,3,Inception,Christopher Nolan,NaN,NaN,148,2010,8.8,74.0
3,4,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,NaN,NaN,178,2001,8.8,92.0
4,5,The Lord of the Rings: The Two Towers,Peter Jackson,NaN,NaN,179,2002,8.8,87.0
...,...,...,...,...,...,...,...,...,...
1744,1745,Radhe,Prabhu Deva,NaN,NaN,113,2021,1.9,NaN
1745,1746,Race 3,Remo D'Souza,NaN,NaN,160,2018,1.9,NaN
1746,1747,Angels Apocalypse,Sean Cain,Enzo Zelocchi,NaN,85,2015,1.7,NaN
1747,1748,The Cost of Deception,Keith English,NaN,NaN,125,2021,1.5,NaN


In [15]:
# scrapping code to collect the data for 2nd csv file.

MoviesName = []
Stars_1 = []
Stars_2 = []
Stars_3 = []
Stars_4 = []
Votes = []
Gross = []
Genre_1 = []
Genre_2 = []
Genre_3 = []
Certification = []
Popularity = []

pages = np.arange(1, 1750, 50)

for page in  tqdm(pages):
    url = "https://www.imdb.com/search/title/?title_type=feature&num_votes=25000,&genres=action&sort=user_rating,desc&start={}&ref_=adv_nxt".format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    movies = soup.find('div', class_="lister-list").find_all('div', class_='lister-item-content')
    #sleep(1)

    for movie in movies:
        movie_name = movie.find('h3', class_="lister-item-header").a.text
        MoviesName.append(movie_name)

        stars = movie.find('p', class_="").get_text(strip=True).split('|')[1].split(',')
        Stars_1.append(stars[0].strip(' Stars: '))
        Stars_2.append(stars[1].strip() if len(stars) >= 2 else np.nan)
        Stars_3.append(stars[2].strip() if len(stars) >= 3 else np.nan)
        Stars_4.append(stars[3].strip() if len(stars) >= 4 else np.nan)

        vote = movie.find('p', class_="sort-num_votes-visible").get_text(strip=True).split('|')[0][6:]
        Votes.append(vote)

        genre = movie.find('span', class_="genre").get_text(strip=True).split(',')

        Genre_1.append(genre[0].strip() if len(genre) >= 1 else np.nan)
        Genre_2.append(genre[1].strip() if len(genre) >= 2 else np.nan)
        Genre_3.append(genre[2].strip() if len(genre) >= 3 else np.nan)

        try:
            gross = movie.find('p', class_="sort-num_votes-visible").get_text(strip=True).split('|')[1][6:].replace('$', '').replace('M', '')
        except IndexError:
            gross = np.nan
        Gross.append(gross)

        try:
            certificate = movie.find('span', class_="certificate").text
        except AttributeError:
            certificate = np.nan
        Certification.append(certificate)
    
        href=movie.find('h3' ,class_="lister-item-header").a.get('href')
        popularity_url=  f"https://www.imdb.com{href}"
        # Connecting to the webdriver
        # Create ChromeDriver instance using WebDriverManager
        driver = webdriver.Chrome()
        # Opening www.imdb.comin in chrome browser
        driver.get(popularity_url)
        try:
            pop=driver.find_element(By.XPATH,'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[3]/a/span/div/div[2]/div[1]')
            Popularity.append(pop.text)
            
        except NoSuchElementException:
            Popularity.append(np.nan)  
        

print(MoviesName,Stars_1,Stars_2,Stars_3,Stars_4,Votes,Gross,Genre_1,Genre_2,Genre_3,Certification,Popularity)
# Close the WebDriver
driver.close()

100%|██████████| 35/35 [4:27:08<00:00, 457.95s/it]  


['The Dark Knight', 'The Lord of the Rings: The Return of the King', 'Inception', 'The Lord of the Rings: The Fellowship of the Ring', 'The Lord of the Rings: The Two Towers', 'The Matrix', 'Star Wars: Episode V - The Empire Strikes Back', 'Soorarai Pottru', 'Star Wars: Episode IV - A New Hope', 'Terminator 2: Judgment Day', 'Seven Samurai', 'Harakiri', 'Sita Ramam', 'Leon', 'Gladiator', 'Spider-Man: Into the Spider-Verse', 'Avengers: Endgame', 'Raiders of the Lost Ark', 'Avengers: Infinity War', 'Oldboy', 'The Dark Knight Rises', 'Aliens', 'Kaithi', 'Asuran', 'Top Gun: Maverick', 'Heat', 'Star Wars: Return of the Jedi', 'Princess Mononoke', 'North by Northwest', 'Kantara', 'Dangal', 'K.G.F: Chapter 2', 'Vikram', 'Raatchasan', 'Shershaah', 'Guardians of the Galaxy Vol. 3', 'Spider-Man: No Way Home', 'Jurassic Park', 'Indiana Jones and the Last Crusade', '1917', 'Kill Bill: Vol. 1', 'Batman Begins', 'Die Hard', 'V for Vendetta', 'Demon Slayer the Movie: Mugen Train', 'Ran', 'Gangs of Wa

In [16]:
# check the length of  collected data
print(len(MoviesName))

1749


Creating CSV File-2

In [18]:
# create the csv file
with open ("D:\Internship\Capstone Project\movie_2.csv" ,'w', newline='', encoding='utf-8') as f:
    writer=csv.DictWriter(f, fieldnames=['MoviesName','Stars_1','Stars_2','Stars_3','Stars_4','Votes','Genre_1','Genre_2','Genre_3','Gross','Certification','Popularity'])
    writer.writeheader()
    
    for Mn,S1,S2,S3,S4,V,Ge_1,Ge_2,Ge_3,G,C,P in zip(MoviesName,Stars_1,Stars_2,Stars_3,Stars_4,Votes,Genre_1,Genre_2,Genre_3,Gross,Certification,Popularity):
        writer.writerow({'MoviesName':Mn, 'Stars_1':S1,'Stars_2':S2,'Stars_3':S3,'Stars_4':S4,'Votes':V, 'Genre_1':Ge_1,'Genre_2':Ge_2,'Genre_3':Ge_3,'Gross':G,'Certification':C,
                        'Popularity':P})

In [19]:
# read the data in csv 
df1=pd.read_csv("D:\Internship\Capstone Project\movie_2.csv")
df1

,MoviesName,Stars_1,Stars_2,Stars_3,Stars_4,Votes,Genre_1,Genre_2,Genre_3,Gross,Certification,Popularity
0,The Dark Knight,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,"2,717,521",Action,Crime,Drama,534.86,12A,117
1,The Lord of the Rings: The Return of the King,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,"1,886,645",Action,Adventure,Drama,377.85,12A,407
2,Inception,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,"2,412,332",Action,Adventure,Sci-Fi,292.58,12A,103
3,The Lord of the Rings: The Fellowship of the Ring,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,"1,915,355",Action,Adventure,Drama,315.54,PG,184
4,The Lord of the Rings: The Two Towers,Elijah Wood,Ian McKellen,Viggo Mortensen,Orlando Bloom,"1,702,927",Action,Adventure,Drama,342.55,12A,788
...,...,...,...,...,...,...,...,...,...,...,...,...
1744,Radhe,lman Khan,Disha Patani,Randeep Hooda,Jackie Shroff,"178,758",Action,Crime,Thriller,NaN,15,NaN
1745,Race 3,Anil Kapoo,Salman Khan,Bobby Deol,Jacqueline Fernandez,"47,520",Action,Crime,Thriller,1.69,12A,NaN
1746,Angels Apocalypse,Enzo Zelocchi,Jana Rochelle,Ryan C.F. Buckley,William Kirkham,"42,912",Action,Fantasy,Sci-Fi,NaN,NaN,NaN
1747,The Cost of Deception,Vivianne Bánovi,András Mózes,Barna Bokor,Gabriella Gubás,"39,414",Action,Crime,Drama,NaN,NaN,NaN


# Phase 2

# Making two tables and corresponding columns provided in the above CSV files in SQLite DBMS. And querying the table(s) in the  SQL workbench / SQLite database

In [20]:
# create a databse with name  Movies.db
conn= sqlite3.connect("D:\Internship\Capstone Project\SQL.db" )

# cursor object
cursor = conn.cursor()

# create Table name = movie_1

table1= """ CREATE TABLE movie_1 (
            Serial_no INTEGER NOT NULL,
            Movie_Name VARCHAR(255) ,
            Director_1 VARCHAR(255),
            Director_2 VARCHAR(255),
            Director_3 VARCHAR(255),
            Duration INTEGER,
            Year TEXT,
            Ratings FLOAT,
            Metascore FLOAT,
            PRIMARY KEY (Serial_no,Movie_Name)
        ); """

cursor.execute(table1)
 
print("Table1 is Ready")

Table1 is Ready


In [21]:
# create Table name = movie_2

table2= """ CREATE TABLE movie_2 (
            MoviesName VARCHAR(250),
            Stars_1 VARCHAR(255),
            Stars_2 VARCHAR(255),
            Stars_3 VARCHAR(255),
            Stars_4 VARCHAR(255),
            Votes INTEGER,
            Genre_1 VARCHAR(25),
            Genre_2 VARCHAR(25),
            Genre_3 VARCHAR(25),
            Gross FLOAT,
            Certification VARCHAR(25),
            Popularity INTEGER,
            FOREIGN KEY (MoviesName) REFERENCES movie_1 (Movie_Name)
        ); """

cursor.execute(table2)
 
print("Table2 is Ready")

Table2 is Ready


In [23]:
# Opening the movie_1.csv file
file = open("D:\Internship\Capstone Project\movie_1.csv" ,encoding='utf-8')
 
# Reading the contents of the

contents = csv.reader(file)
next(contents)
# SQL query to insert data into the table
insert_records = " INSERT INTO movie_1 (Serial_no, Movie_Name, Director_1,Director_2,Director_3, Duration, Year, Ratings, Metascore) VALUES(?,?,?,?,?,?,?,?,?)"
 
# Importing the contents of the file
# into our person table
cursor.executemany(insert_records, contents)
conn.commit()

# SQL query to retrieve all data from the table
select_all = "SELECT * FROM movie_1"
rows = cursor.execute(select_all).fetchall()

cursor.lastrowid

1749

In [24]:
# Opening the movie_2.csv file
file = open("D:\Internship\Capstone Project\movie_2.csv" , encoding='utf-8')
 
# Reading the contents of the

contents = csv.reader(file)
next(contents)
# SQL query to insert data into the table
insert_records = " INSERT INTO movie_2 (MoviesName, Stars_1,Stars_2,Stars_3,Stars_4, Votes, Genre_1,Genre_2,Genre_3, Gross, Certification, Popularity) VALUES(?,?,?,?,?,?,?,?,?,?,?,?)"
 
# Importing the contents of the file into our table
cursor.executemany(insert_records, contents)
conn.commit()

# SQL query to retrieve all data from the table
select_all = "SELECT * FROM movie_2"
rows = cursor.execute(select_all).fetchall()
conn.close()

cursor.lastrowid

1749

# Table 1

1) Display all the details of movies created by directors Christopher and Matt Reeves.

In [25]:
 
conn= sqlite3.connect("D:\Internship\Capstone Project\SQL.db" )

# cursor object
mycursor = conn.cursor()
mycursor.execute("SELECT * FROM movie_1 WHERE Director_1  in ('Christopher Nolan','Matt Reev');")
myresult=mycursor.fetchall()

for x in myresult:
    print(x)

(1, 'The Dark Knight', 'Christopher Nolan', 'nan', 'nan', 152, '2008', 9.0, 84.0)
(3, 'Inception', 'Christopher Nolan', 'nan', 'nan', 148, '2010', 8.8, 74.0)
(21, 'The Dark Knight Rises', 'Christopher Nolan', 'nan', 'nan', 164, '2012', 8.4, 78.0)
(42, 'Batman Begins', 'Christopher Nolan', 'nan', 'nan', 140, '2005', 8.2, 70.0)
(130, 'The Batman', 'Matt Reev', 'nan', 'nan', 176, '2022', 7.8, 72.0)
(138, 'Dunkirk', 'Christopher Nolan', 'nan', 'nan', 106, '2017', 7.8, 94.0)
(224, 'Dawn of the Planet of the Apes', 'Matt Reev', 'nan', 'nan', 130, '2014', 7.6, 79.0)
(305, 'War for the Planet of the Apes', 'Matt Reev', 'nan', 'nan', 140, '2017', 7.4, 82.0)
(321, 'Tenet', 'Christopher Nolan', 'nan', 'nan', 150, '2020', 7.3, 69.0)
(503, 'Cloverfield', 'Matt Reev', 'nan', 'nan', 85, '2008', 7.0, 64.0)


2) Display all the details of movies with a duration of 140 minutes to 190 minutes.

In [26]:

mycursor.execute("SELECT * FROM movie_1 WHERE Duration BETWEEN 140 and 190;")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

(1, 'The Dark Knight', 'Christopher Nolan', 'nan', 'nan', 152, '2008', 9.0, 84.0)
(3, 'Inception', 'Christopher Nolan', 'nan', 'nan', 148, '2010', 8.8, 74.0)
(4, 'The Lord of the Rings: The Fellowship of the Ring', 'Peter Jackson', 'nan', 'nan', 178, '2001', 8.8, 92.0)
(5, 'The Lord of the Rings: The Two Towers', 'Peter Jackson', 'nan', 'nan', 179, '2002', 8.8, 87.0)
(8, 'Soorarai Pottru', 'Sudha Kongara', 'nan', 'nan', 153, '2020', 8.7, 'nan')
(13, 'Sita Ramam', 'Hanu Raghavapud', 'nan', 'nan', 163, '2022', 8.6, 'nan')
(15, 'Gladiator', 'Ridley S', 'nan', 'nan', 155, '2000', 8.5, 67.0)
(17, 'Avengers: Endgame', 'Anthony Ru', 'Joe Russo', 'nan', 181, '2019', 8.4, 78.0)
(19, 'Avengers: Infinity War', 'Anthony Ru', 'Joe Russo', 'nan', 149, '2018', 8.4, 68.0)
(21, 'The Dark Knight Rises', 'Christopher Nolan', 'nan', 'nan', 164, '2012', 8.4, 78.0)
(23, 'Kaithi', 'Lokesh Kanagaraj', 'nan', 'nan', 145, '2019', 8.4, 'nan')
(24, 'Asuran', 'Vetrimaaran', 'nan', 'nan', 141, '2019', 8.4, 'nan')
(

3) Display all details of movies with ratings above 7 in ascending order.

In [27]:

mycursor.execute("SELECT * FROM movie_1 where Ratings >7.0 ORDER by Ratings;")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

(430, 'Fast & Furious 7', 'James Wan', 'nan', 'nan', 137, '2015', 7.1, 67.0)
(431, 'The Mummy', 'Stephen Somm', 'nan', 'nan', 124, '1999', 7.1, 48.0)
(432, 'Mission: Impossible', 'Brian De Palma', 'nan', 'nan', 110, '1996', 7.1, 59.0)
(433, 'Free Guy', 'Shawn Levy', 'nan', 'nan', 115, '2021', 7.1, 62.0)
(434, 'Wrath of Man', 'Guy Ritch', 'nan', 'nan', 119, '2021', 7.1, 57.0)
(435, 'Tropic Thunder', 'Ben Still', 'nan', 'nan', 107, '2008', 7.1, 71.0)
(436, 'Prey', 'an Trachtenberg', 'nan', 'nan', 100, '2022', 7.1, 71.0)
(437, 'Man of Steel', 'Zack Snyd', 'nan', 'nan', 143, '2013', 7.1, 55.0)
(438, 'Everest', 'Baltasar Kormáku', 'nan', 'nan', 121, '2015', 7.1, 64.0)
(439, 'Blade', 'Stephen Norrington', 'nan', 'nan', 120, '1998', 7.1, 47.0)
(440, 'Shooter', 'Antoine Fuqua', 'nan', 'nan', 124, '2007', 7.1, 53.0)
(441, 'Batman Returns', 'Tim Burton', 'nan', 'nan', 126, '1992', 7.1, 68.0)
(442, 'Highlander', 'Russell Mulcahy', 'nan', 'nan', 116, '1986', 7.1, 24.0)
(443, 'Iron Man Three', 'Sha

4) Display all movie names in descending order.

In [28]:

mycursor.execute("SELECT Movie_Name FROM movie_1 ORDER by Movie_Name DESC;")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('Æon Flux',)
('xXx²: The Next Level',)
('xXx: Return of Xander Cage',)
('xXx',)
('Zoolander 2',)
('Zombieland: Double Tap',)
('Zombieland',)
('Zatoichi: The Blind Swordsman',)
('Zathura: A Space Adventure',)
("Zack Snyder's Justice League",)
('Your Highness',)
('Young Guns II: Blaze of Glory',)
('Young Guns',)
('You Only Live Twice',)
("You Don't Mess with the Zohan",)
('Yojimbo',)
('X: First Class',)
('X-Men: The Last Stand',)
('X-Men: Days of Future Past',)
('X-Men: Dark Phoenix',)
('X-Men: Apocalypse',)
('X-Men Origins: Wolverine',)
('X-Men 2',)
('X-Men',)
('Wyatt Earp',)
('Wrongfully Accused',)
('Wrath of the Titans',)
('Wrath of Man',)
('World War Z',)
('Wonder Woman 1984',)
('Wonder Woman',)
('Wolfwalkers',)
('Without Remorse',)
('Windtalkers',)
("Willy's Wonderland",)
('Willow',)
('Wild Wild West',)
('Wild Target',)
('Wild Hogs',)
('Wild Card',)
('Who Am I?',)
('Whiteout',)
('White House Down',)
('White Heat',)
('Where Eagles Dare',)
('What Happened to Monday',)
('Westworld',)


5) Display movie name starts with ‘P’ and their rating is greater than 7

In [29]:

mycursor.execute("SELECT Movie_Name FROM movie_1 WHERE Movie_Name like 'P%' AND Ratings>7.0;")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('Princess Mononoke',)
('Paan Singh Tomar',)
('Pirates of the Caribbean: The Curse of the Black Pearl',)
('Predator',)
('Ponniyin Selvan',)
('Pushpa: The Rise - Part 1',)
('Police Story',)
('Predestination',)
("Pirates of the Caribbean: Dead Man's Chest",)
('Patriots Day',)
('Point Break',)
('Prey',)
("Pirates of the Caribbean: At World's End",)
('Planet Terror',)
('Payback',)


# Table 2

1) Display all movie names with star Arnold Schwarzenegger in ascending order.

In [30]:

mycursor.execute("""SELECT MoviesName FROM movie_2
WHERE (Stars_1 in ('Arnold Schwarzenegge','Arnold Schwarzenegger') or Stars_2 in ('Arnold Schwarzenegge','Arnold Schwarzenegger'));
""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('Terminator 2: Judgment Day',)
('The Terminator',)
('Predator',)
('Total Recall',)
('True Lies',)
('Conan the Barbarian',)
('The Running Man',)
('Commando',)
('Escape Plan',)
('Last Action Hero',)
('Terminator Genisys',)
('The Last Stand',)
('Terminator 3: Rise of the Machines',)
('Kindergarten Cop',)
('Terminator: Dark Fate',)
('Eraser',)
('Red Heat',)
('Conan the Destroyer',)
('The 6th Day',)
('End of Days',)
('Sabotage',)
('Raw Deal',)
('Collateral Damage',)
('Red Sonja',)
('Batman & Robin',)


2) Display all details of the movie with the highest number of votes.

In [31]:

mycursor.execute("SELECT * FROM movie_2 WHERE Votes=(SELECT max(Votes) FROM movie_2);")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('The American', 'George Clooney', 'Paolo Bonacelli', 'Violante Placido', 'Irina Björklund', '99,905', 'Action', 'Crime', 'Drama', 35.61, '15', '2,657')


3) Display movie names with gross collections in descending order.

In [32]:

mycursor.execute("SELECT MoviesName,Gross FROM movie_2 WHERE Gross !='nan' ORDER by Gross DESC;")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('Star Wars: Episode VII - The Force Awakens', 936.66)
('Avengers: Endgame', 858.37)
('Spider-Man: No Way Home', 804.75)
('Avatar', 760.51)
('Top Gun: Maverick', 718.73)
('Black Panther', 700.06)
('Avengers: Infinity War', 678.82)
('Avatar: The Way of Water', 659.68)
('Jurassic World', 652.27)
('Avengers Assemble', 623.28)
('Star Wars: Episode VIII - The Last Jedi', 620.18)
('The Incredibles 2', 608.58)
('The Dark Knight', 534.86)
('Rogue One', 532.18)
('Star Wars: The Rise of Skywalker', 515.2)
('Star Wars: Episode I - The Phantom Menace', 474.54)
('Avengers: Age of Ultron', 459.01)
('Black Panther: Wakanda Forever', 453.72)
('The Dark Knight Rises', 448.14)
('Captain Marvel', 426.83)
('The Hunger Games: Catching Fire', 424.67)
("Pirates of the Caribbean: Dead Man's Chest", 423.32)
('Jurassic World: Fallen Kingdom', 417.72)
('Wonder Woman', 412.56)
('Doctor Strange in the Multiverse of Madness', 411.33)
('Iron Man Three', 409.01)
('Captain America: Civil War', 408.08)
('The Hunger Gam

4) Display the gross collection of movies with the star Arnold.

In [33]:

mycursor.execute("""SELECT MoviesName,Gross FROM movie_2
WHERE Stars_1 like '%Arnold%' or  Stars_2 like '%Arnold%' or  Stars_3 like '%Arnold%' or  Stars_4 like '%Arnold%';""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('Terminator 2: Judgment Day', 204.84)
('The Terminator', 38.4)
('Predator', 59.74)
('Total Recall', 119.39)
('True Lies', 146.28)
('The Mummy', 155.25)
('Conan the Barbarian', 39.57)
('The Running Man', 38.12)
('Commando', 35.1)
('Escape Plan', 25.13)
('Last Action Hero', 50.02)
('The Mummy Returns', 202.02)
('Terminator Genisys', 89.76)
('The Last Stand', 12.05)
('Terminator 3: Rise of the Machines', 150.37)
('Kindergarten Cop', 91.46)
('Terminator: Dark Fate', 62.25)
('Eraser', 101.3)
('Red Heat', 34.99)
('Conan the Destroyer', 31.04)
('The 6th Day', 34.6)
('End of Days', 66.89)
('Sabotage', 10.51)
('Raw Deal', 16.21)
('Collateral Damage', 40.08)
('Red Sonja', 6.95)
('Batman & Robin', 107.33)


5) Display all details of movies with comedy and action genres.

In [34]:

mycursor.execute("""SELECT * FROM movie_2
WHERE  Genre_1 in ('Action','Comedy') AND Genre_2 in ('Action','Comedy') or Genre_3 in ('Action','Comedy');""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('Guardians of the Galaxy Vol. 3', 'Chris P', 'Chukwudi Iwuji', 'Bradley Cooper', 'Pom Klementieff', '146,352', 'Action', 'Adventure', 'Comedy', 'nan', '12A', 3)
('Gangs of Wasseypur', 'Manoj Bajpayee', 'Nawazuddin Siddiqui', 'Tigmanshu Dhulia', 'Richa Chadha', '99,325', 'Action', 'Comedy', 'Crime', 'nan', '18', '4,011')
('Sherlock Jr.', 'Buster Keaton', 'Kathryn McGuire', 'Joe Keaton', 'Erwin Connelly', '52,431', 'Action', 'Comedy', 'Romance', 0.98, 'U', 'nan')
('Lock, Stock and Two Smoking Barrels', 'Jason Flemyng', 'Dexter Fletcher', 'Nick Moran', 'Jason Statham', '595,108', 'Action', 'Comedy', 'Crime', 3.9, '18', 894)
('Sholay', 'njeev Kum', 'Dharmendra', 'Amitabh Bachchan', 'Amjad Khan', '57,063', 'Action', 'Adventure', 'Comedy', 'nan', 'PG', 'nan')
('Bajrangi Bhaijaan', 'lman Khan', 'Harshaali Malhotra', 'Nawazuddin Siddiqui', 'Kareena Kapoor', '91,904', 'Action', 'Adventure', 'Comedy', 8.18, '12A', 'nan')
('The General', 'Buster Keaton', 'Marion Mack', 'Glen Cavender', 'Jim Farl

# Make subquery :

1) Display all details from both tables where movie names are the same.

In [35]:

mycursor.execute("""SELECT * FROM movie_1 
JOIN movie_2
on movie_1.Movie_Name=movie_2.MoviesName
WHERE movie_1.Movie_Name in (select Movie_Name from (SELECT Movie_Name , count(*) as count from movie_1
group by Movie_Name
HAVING count>1)) ;""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

(20, 'Oldboy', 'Park Chan-wook', 'nan', 'nan', 120, '2003', 8.4, 77.0, 'Oldboy', 'Choi Min-sik', 'Yoo Ji-tae', 'Kang Hye-jeong', 'Kim Byeong-Ok', '596,202', 'Action', 'Drama', 'Mystery', 0.71, '18', 314)
(20, 'Oldboy', 'Park Chan-wook', 'nan', 'nan', 120, '2003', 8.4, 77.0, 'Oldboy', 'Josh Brolin', 'Elizabeth Olsen', 'Samuel L. Jackson', 'Sharlto Copley', '78,217', 'Action', 'Drama', 'Mystery', 2.19, '18', '1,320')
(82, 'Dune', 'nis Villeneuv', 'nan', 'nan', 155, '2021', 8.0, 74.0, 'Dune', 'Kyle MacLachlan', 'Virginia Madsen', 'Francesca Annis', 'Leonardo Cimino', '165,724', 'Action', 'Adventure', 'Sci-Fi', 30.93, '12', 399)
(82, 'Dune', 'nis Villeneuv', 'nan', 'nan', 155, '2021', 8.0, 74.0, 'Dune', 'Timothée Chalame', 'Rebecca Ferguson', 'Zendaya', 'Oscar Isaac', '677,751', 'Action', 'Adventure', 'Drama', 108.33, '12A', 66)
(118, 'Ghost in the Shell', 'Mamoru Osh', 'nan', 'nan', 83, '1995', 7.9, 76.0, 'Ghost in the Shell', 'Atsuko Tanak', 'Iemasa Kayumi', 'Akio Ôtsuka', 'Kôichi Yamade

2) Display all movie names, Director, ratings, and gross collection where the genre is action.

In [36]:

mycursor.execute("""SELECT Movie_Name,Director_1,Director_2,Director_3,Ratings,movie_2.Gross FROM movie_1 
JOIN movie_2
on movie_1.Movie_Name=movie_2.MoviesName
WHERE Genre_1='Action' or Genre_2='Action' or Genre_3='Action';""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

('The Dark Knight', 'Christopher Nolan', 'nan', 'nan', 9.0, 534.86)
('The Lord of the Rings: The Return of the King', 'Peter Jackson', 'nan', 'nan', 9.0, 377.85)
('Inception', 'Christopher Nolan', 'nan', 'nan', 8.8, 292.58)
('The Lord of the Rings: The Fellowship of the Ring', 'Peter Jackson', 'nan', 'nan', 8.8, 315.54)
('The Lord of the Rings: The Two Towers', 'Peter Jackson', 'nan', 'nan', 8.8, 342.55)
('The Matrix', 'Lana Wachowsk', 'Lilly Wachowski', 'nan', 8.7, 171.48)
('Star Wars: Episode V - The Empire Strikes Back', 'Irvin Kershn', 'nan', 'nan', 8.7, 290.48)
('Soorarai Pottru', 'Sudha Kongara', 'nan', 'nan', 8.7, 'nan')
('Star Wars: Episode IV - A New Hope', 'George Luca', 'nan', 'nan', 8.6, 322.74)
('Terminator 2: Judgment Day', 'James Cameron', 'nan', 'nan', 8.6, 204.84)
('Seven Samurai', 'Akira Kurosawa', 'nan', 'nan', 8.6, 0.27)
('Harakiri', 'Masaki Kobayash', 'nan', 'nan', 8.6, 'nan')
('Sita Ramam', 'Hanu Raghavapud', 'nan', 'nan', 8.6, 'nan')
('Leon', 'Luc Besson', 'nan',

3) Display all details from both tables with the highest gross collection.

In [37]:

mycursor.execute("""SELECT * FROM movie_1 
JOIN movie_2
on movie_1.Movie_Name=movie_2.MoviesName
WHERE movie_2.Gross=(SELECT max(movie_2.Gross) FROM movie_2 WHERE movie_2.Gross!='nan');""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

(136, 'Star Wars: Episode VII - The Force Awakens', 'J.J. Abram', 'nan', 'nan', 138, '2015', 7.8, 80.0, 'Star Wars: Episode VII - The Force Awakens', 'Daisy Ridley', 'John Boyega', 'Oscar Isaac', 'Domhnall Gleeson', '947,209', 'Action', 'Adventure', 'Sci-Fi', 936.66, '12A', 455)


4) Display all details from both tables with the highest ratings.

In [38]:

mycursor.execute("""SELECT * FROM movie_1 
JOIN movie_2
on movie_1.Movie_Name=movie_2.MoviesName
WHERE movie_1.Ratings=(SELECT max(movie_1.Ratings) FROM movie_1);""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

(1, 'The Dark Knight', 'Christopher Nolan', 'nan', 'nan', 152, '2008', 9.0, 84.0, 'The Dark Knight', 'Christian Bale', 'Heath Ledger', 'Aaron Eckhart', 'Michael Caine', '2,717,521', 'Action', 'Crime', 'Drama', 534.86, '12A', 117)
(2, 'The Lord of the Rings: The Return of the King', 'Peter Jackson', 'nan', 'nan', 201, '2003', 9.0, 94.0, 'The Lord of the Rings: The Return of the King', 'Elijah Wood', 'Viggo Mortensen', 'Ian McKellen', 'Orlando Bloom', '1,886,645', 'Action', 'Adventure', 'Drama', 377.85, '12A', 407)


5) Display all details from both tables with the lowest gross collection and lowest ratings.

In [39]:

mycursor.execute("""SELECT * FROM movie_1 
JOIN movie_2
on movie_1.Movie_Name=movie_2.MoviesName
WHERE movie_2.Gross=(SELECT min(movie_2.Gross) FROM movie_2)
ORDER by movie_1.Ratings LIMIT 1;""")
myresult= mycursor.fetchall()

for x in myresult:
    print(x)

(1481, 'Wild Card', 'Simon W', 'nan', 'nan', 92, '2015', 5.6, 40.0, 'Wild Card', 'Jason Statham', 'Michael Angarano', 'Dominik Garcia', 'Milo Ventimiglia', '57,742', 'Action', 'Crime', 'Drama', 0.0, '15', '3,208')
